In [2]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import data_tools

In [3]:
from datetime import datetime
import pickle

import lightgbm as lgb
import pandas as pd
import numpy as np

SEED = 42

In [9]:
X = pd.read_pickle("../data/intermediate/X.pkl")
Y = pd.read_pickle("../data/intermediate/Y.pkl")

In [6]:
# need to convert all categorical to numbers for sklearn SVM
# first convert the nans in our categorical back to "?", will use as category
X = X.apply(lambda x: x.cat.add_categories("?").fillna("?") if x.dtype.name == 'category' and not "?" in x.cat.categories.values else x)

In [16]:
X = pd.get_dummies(data=X, columns=X.columns[X.dtypes == 'category'], drop_first=True)

### Feature Selection

In [17]:
field_descriptions_fpath = data_tools.field_descriptions_fpath
fields = data_tools.parse_field_descriptions(field_descriptions_fpath)

In [18]:
# Use this to select features
# currently using all features so ignore this, probably want to do this before we extend the categorical vars above to make it easier
selected_features = None

2960

### Validation

In [19]:
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import svm

In [25]:
hyper_params = {
    'C': [0.1, 1, 10],
    'tol': [0.001, 0.01, 0.1],
    'class_weight': [None, 'balanced'], # balanced option helps with class frequencies, might be worth looking into
    'kernel': ['linear', 'rbf'],
    'random_state' : [SEED]
}
svc = svm.SVC()
search = GridSearchCV(estimator = svc, param_grid=hyper_params, refit = True, cv = 5)
search_results = search.fit(X, Y)

In [26]:
search_results.best_params_

{'C': 1,
 'class_weight': 'balanced',
 'kernel': 'linear',
 'random_state': 42,
 'tol': 0.001}

In [27]:
search_results.best_score_

0.9872496303266569